In [ ]:
##########

## Project: Amazon Reviews Sentiment Analysis with NLTK's VADER

##########

In [ ]:
# Vader sums up sentiment scores for each word to come up with sentiment and intensity of sentiment scores

In [ ]:
# As we will see, VADER can detect positive sentiment well, but it fails to catch sarcasm in a sample text,
# It also fails to distinguish between positive and negative sentiments within the same text (in our case a product review) at hand

# Part 1: Playing with VADER functionality to get Familiar with It

In [ ]:
# load packages

In [ ]:
import nltk

In [ ]:
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [ ]:
sid = SentimentIntensityAnalyzer()

In [ ]:
# now test on sample text

In [ ]:
a = 'This is a good movie'

In [ ]:
sid.polarity_scores(a)

{'neg': 0.0, 'neu': 0.508, 'pos': 0.492, 'compound': 0.4404}

In [ ]:
a = 'This was the best, most awesome movie EVER MADE!!!'

In [ ]:
sid.polarity_scores(a)

{'neg': 0.0, 'neu': 0.425, 'pos': 0.575, 'compound': 0.8877}

{'neg': 0.0, 'neu': 0.425, 'pos': 0.575, 'compound': 0.8877}

In [ ]:
a = 'This was the WORST movie that ever disgraced the screen.'

In [ ]:
sid.polarity_scores(a)

{'neg': 0.495, 'neu': 0.505, 'pos': 0.0, 'compound': -0.8331}

In [ ]:
# Note the variation in scores allocated amongst the 3 texts we fed into VADER

# Part 2: Load and Explore Our Reviews Dataset

In [ ]:
# load more packages

In [ ]:
import pandas as pd

In [ ]:
# read in the dataset, note that we already have the labels which means we can actually measure success of our sentiment scores at the end

In [ ]:
df = pd.read_csv('amazonreviews.tsv',sep='\t')

In [ ]:
df.head()

,label,review
0,pos,Stuning even for the non-gamer: This sound tra...
1,pos,The best soundtrack ever to anything.: I'm rea...
2,pos,Amazing!: This soundtrack is my favorite music...
3,pos,Excellent Soundtrack: I truly like this soundt...
4,pos,"Remember, Pull Your Jaw Off The Floor After He..."


In [ ]:
# get more info about dataset

In [ ]:
df.columns

Index(['label', 'review'], dtype='object')

In [ ]:
df['label'].value_counts()

neg    5097
pos    4903
Name: label, dtype: int64

In [ ]:
df.describe()

,label,review
count,10000,10000
unique,2,10000
top,neg,Stuning even for the non-gamer: This sound tra...
freq,5097,1


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   label   10000 non-null  object
 1   review  10000 non-null  object
dtypes: object(2)
memory usage: 156.4+ KB


In [ ]:
# while above initially shows there are no missing values, it might be deceiving since we may have reviews populated with a space? let's investigate

In [ ]:
blanks = []
for i,lb,rv in df.itertuples():
  # index,label,review
  if type(rv)==str:
    if rv.isspace():
      blanks.append(i)

In [ ]:
blanks

[]

In [ ]:
# ok in this case we have 0 blank reviews

In [ ]:
# Now let's visualize the first review

In [ ]:
df.iloc[0]['review']

'Stuning even for the non-gamer: This sound track was beautiful! It paints the senery in your mind so well I would recomend it even to people who hate vid. game music! I have played the game Chrono Cross but out of all of the games I have ever played it has the best music! It backs away from crude keyboarding and takes a fresher step with grate guitars and soulful orchestras. It would impress anyone who cares to listen! ^_^'

In [ ]:
# and check it's sentiment analysis scores

In [ ]:
sid.polarity_scores(df.iloc[0]['review'])

{'neg': 0.088, 'neu': 0.669, 'pos': 0.243, 'compound': 0.9454}

# Part 2: Apply Sentiment Analysis to Our Reviews Dataset

In [ ]:
# let's create a variable for the VADER scores

In [ ]:
df['scores'] = df['review'].apply(lambda review: sid.polarity_scores(review))

In [ ]:
df.head()

,label,review,scores
0,pos,Stuning even for the non-gamer: This sound tra...,"{'neg': 0.088, 'neu': 0.669, 'pos': 0.243, 'co..."
1,pos,The best soundtrack ever to anything.: I'm rea...,"{'neg': 0.018, 'neu': 0.837, 'pos': 0.145, 'co..."
2,pos,Amazing!: This soundtrack is my favorite music...,"{'neg': 0.04, 'neu': 0.692, 'pos': 0.268, 'com..."
3,pos,Excellent Soundtrack: I truly like this soundt...,"{'neg': 0.09, 'neu': 0.615, 'pos': 0.295, 'com..."
4,pos,"Remember, Pull Your Jaw Off The Floor After He...","{'neg': 0.0, 'neu': 0.746, 'pos': 0.254, 'comp..."


In [ ]:
# extract the compound score component into a new variable

In [ ]:
df['compound'] = df['scores'].apply(lambda d: d['compound'])

In [ ]:
df.head()

,label,review,scores,compound
0,pos,Stuning even for the non-gamer: This sound tra...,"{'neg': 0.088, 'neu': 0.669, 'pos': 0.243, 'co...",0.9454
1,pos,The best soundtrack ever to anything.: I'm rea...,"{'neg': 0.018, 'neu': 0.837, 'pos': 0.145, 'co...",0.8957
2,pos,Amazing!: This soundtrack is my favorite music...,"{'neg': 0.04, 'neu': 0.692, 'pos': 0.268, 'com...",0.9858
3,pos,Excellent Soundtrack: I truly like this soundt...,"{'neg': 0.09, 'neu': 0.615, 'pos': 0.295, 'com...",0.9814
4,pos,"Remember, Pull Your Jaw Off The Floor After He...","{'neg': 0.0, 'neu': 0.746, 'pos': 0.254, 'comp...",0.9781


In [ ]:
# create a categorical variable to classify each review as positive or negative based on its previously-computed compound score (>= 0 is synonymous to a positive review)

In [ ]:
df['comp_score'] = df['compound'].apply(lambda score: 'pos' if score >=0 else 'neg')

In [ ]:
df.head()

,label,review,scores,compound,comp_score
0,pos,Stuning even for the non-gamer: This sound tra...,"{'neg': 0.088, 'neu': 0.669, 'pos': 0.243, 'co...",0.9454,pos
1,pos,The best soundtrack ever to anything.: I'm rea...,"{'neg': 0.018, 'neu': 0.837, 'pos': 0.145, 'co...",0.8957,pos
2,pos,Amazing!: This soundtrack is my favorite music...,"{'neg': 0.04, 'neu': 0.692, 'pos': 0.268, 'com...",0.9858,pos
3,pos,Excellent Soundtrack: I truly like this soundt...,"{'neg': 0.09, 'neu': 0.615, 'pos': 0.295, 'com...",0.9814,pos
4,pos,"Remember, Pull Your Jaw Off The Floor After He...","{'neg': 0.0, 'neu': 0.746, 'pos': 0.254, 'comp...",0.9781,pos


# Part 3: Evaluate Resulting Computed Scores relative to pre-Existing Labels

In [ ]:
from sklearn.metrics import accuracy_score, classification_report,confusion_matrix

In [ ]:
accuracy_score(df['label'],df['comp_score'])

0.7097

In [ ]:
# better than random guessing which is 0.5 btw in this case since we have 2 categories: Positive or Negative!

In [ ]:
# now print classification report and confusion matrix to look into the specificities of evaluation metrics

In [ ]:
print(classification_report(df['label'],df['comp_score']))

              precision    recall  f1-score   support

         neg       0.86      0.52      0.64      5097
         pos       0.64      0.91      0.75      4903

    accuracy                           0.71     10000
   macro avg       0.75      0.71      0.70     10000
weighted avg       0.75      0.71      0.70     10000



In [ ]:
print(confusion_matrix(df['label'],df['comp_score']))

[[2629 2468]
 [ 435 4468]]


In [ ]:
# VADER classified reviews correctly 71% of time.
# However, just looking at the precision and recall scores across positive and negative reviews shows a high score for positive reviews 
# but huge underperformance for negative reviews
# Therefore, NLTK's VADER can't be taken seriously as a state of the art sentiment analysis tool!